# Carga y análisis del dataset

Instalar librerías necesarias (si no están instaladas)

In [ ]:
!pip install pandas openpyxl matplotlib seaborn --quiet
!pip install datasets --quiet
!pip install numpy==1.26.4 --quiet # (versión anterior)

# Después de ejecutar estos comandos es necesario reiniciar el kernel

Importar librerías


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import EarlyStoppingCallback
from datasets import Dataset
import torch

Carga del dataset y normalización

In [ ]:
ruta = "/content/Dataset_Completo.xlsx"
df = pd.read_excel(ruta)

# Limpieza y normalización de la columna 'Category'
df['Category'] = df['Category'].astype(str).str.strip().str.lower()

# Mapeo de etiquetas: 'true' → 0 y 'fake' → 1
df['label'] = df['Category'].map({'true': 0, 'fake': 1})

# Verificación
print("Valores únicos en 'Category':", df['Category'].unique())
print("Valores únicos en 'label':", df['label'].unique())
print("\nDistribución de clases:")
print(df['label'].value_counts())

Distribución de clases

In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(data=df, x='label')
plt.title("Distribución de clases (0 = verdadera, 1 = falsa)")
plt.xlabel("Etiqueta")
plt.ylabel("Número de ejemplos")
plt.show()

Convertimos a texto y calculamos la longitud

In [ ]:
df['text_len'] = df['Text'].astype(str).apply(len)
print("\nLongitud media de los textos:", df['text_len'].mean())

Gráfico distribución de la longitud de los textos

In [ ]:
plt.figure(figsize=(8,4))
sns.histplot(df['text_len'], bins=40, kde=True)
plt.title("Distribución de la longitud de los textos")
plt.xlabel("Número de caracteres")
plt.ylabel("Frecuencia")
plt.show()

Mostrar primeros registros del dataset

In [ ]:
# Mostrar los primeros registros del dataset
print("\nPrimeros registros del dataset:")
df.head()

Datos nulos y datos faltantes

In [ ]:
# Comprobar si existen registros con al menos un campo en blanco o nulo
registros_con_faltantes = df[df.isnull().any(axis=1)]
total_con_faltantes = registros_con_faltantes.shape[0]

# Mostrar resultado general
print(f"¿Existen registros con datos faltantes?: {total_con_faltantes > 0}")
print(f"Total de registros con al menos un campo en blanco: {total_con_faltantes}\n")

# Mostrar el número de valores nulos por columna
print("Valores nulos por columna:")
print(df.isnull().sum())

In [ ]:
# Visualizar los primeros registros con campos faltantes
if total_con_faltantes > 0:
    print("\nPrimeros registros con campos faltantes:")
    print(registros_con_faltantes.head())

# Entrenamiento de BETO con el texto completo


In [ ]:
from sklearn.model_selection import train_test_split

# 1. Usamos 'text', 'topic', 'source' y 'headline' como entrada. El campo LINK no lo utilizamos para el entrenamiento
df_texto = df[['Text', 'Topic', 'Source', 'Headline', 'label']].dropna().copy()

# Renombramos todas las columnas
df_texto.rename(columns={
    'Text': 'text',
    'Topic': 'topic',
    'Source': 'source',
    'Headline': 'headline'
}, inplace=True)

# 2. División del dataset en entrenamiento y prueba
train_df, test_df = train_test_split(df_texto, test_size=0.2, stratify=df_texto['label'], random_state=42)

# 3. Conversión a datasets de Hugging Face
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# 4. Tokenizador y tokenización
tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

# 5. Formateo
tokenized_train.set_format("torch", columns=["input_ids", "attention_mask", "label"])
tokenized_test.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# 6. Modelo BETO
model = BertForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-cased", num_labels=2)

# 7. Configuración de entrenamiento
training_args = TrainingArguments(
    output_dir="./results_texto_completo",
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    eval_strategy="epoch",  # Para evaluar cada época
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    logging_steps=10,
    report_to="none"
)

# 8 Preparación metricas entrenamiento
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

# 9. Entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],  # Espera 2 épocas sin mejorar
)

# 10. Entrenar el modelo
trainer.train()

Guardar el modelo entrenado con textos (lo usaremos con LIME).

In [ ]:
# Guardar modelo y tokenizer tras el entrenamiento
trainer.save_model("./modelo_beto_fake_news")
tokenizer.save_pretrained("./modelo_beto_fake_news")

# Evaluación del modelo con texto completo

In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix, roc_auc_score, roc_curve
)

# 1. Obtener predicciones
predictions = trainer.predict(tokenized_test)
preds = np.argmax(predictions.predictions, axis=1)
labels = predictions.label_ids

# 2. Métricas principales
accuracy = accuracy_score(labels, preds)
precision = precision_score(labels, preds)
recall = recall_score(labels, preds)
f1 = f1_score(labels, preds)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

# 3. Reporte completo por clase
print("\nReporte completo:")
print(classification_report(labels, preds, target_names=["True", "Fake"]))

# 4. Matriz de confusión
cm = confusion_matrix(labels, preds)
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["True", "Fake"], yticklabels=["True", "Fake"])
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.title('Matriz de Confusión - Texto completo (BETO)')
plt.show()

# 5. Curva ROC y AUC
probs = torch.nn.functional.softmax(torch.tensor(predictions.predictions), dim=1)[:, 1].numpy()
auc = roc_auc_score(labels, probs)
fpr, tpr, _ = roc_curve(labels, probs)

plt.figure(figsize=(7, 5))
plt.plot(fpr, tpr, label=f'ROC curve (AUC = {auc:.4f})')
plt.plot([0, 1], [0, 1], 'k--', label='Clasificador aleatorio')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('Curva ROC - Modelo con texto completo (BETO)')
plt.legend(loc='lower right')
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()



---



# Interpretabilidad con LIME

Instalar LIME

In [ ]:
!pip install lime --quiet

In [ ]:
from transformers import pipeline

pipe = pipeline(
    "text-classification",
    model="./modelo_beto_fake_news",
    tokenizer="./modelo_beto_fake_news",
    return_all_scores=True
)

Importaciones y función predictiva

In [ ]:
from lime.lime_text import LimeTextExplainer

# Definimos los nombres de las clases
class_names = ['True', 'Fake']

# Función para LIME: devuelve solo las probabilidades de clase
def predict_lime(texts):
    outputs = pipe(texts)
    return np.array([[s['score'] for s in o] for o in outputs])

Inicializar el explicador y elegir un texto

In [ ]:
explainer = LimeTextExplainer(class_names=class_names, random_state=42)

# Seleccionamos un texto del dataset
texto_ejemplo = df['Text'].dropna().astype(str).iloc[0][:1000]  # truncado por seguridad

Generar la explicación y visualizarla

In [ ]:
exp = explainer.explain_instance(texto_ejemplo, predict_lime, num_features=10)

# Mostrar en notebook
exp.show_in_notebook(text=True)

Guardar la explicación LIME como archivo HTML

In [ ]:
# Guardar la explicación como archivo HTML
exp.save_to_file('lime_explicacion_BETO.html')



---



# Interpretabilidad con SHAP

Aunque las explicaciones SHAP se generan sobre un modelo base multilingüe, la interpretación general puede extrapolarse razonablemente al comportamiento del modelo fine-tuned entrenado en este trabajo, ya que ambos comparten la misma arquitectura.

Instalar librerías necesarias

In [ ]:
!pip install shap scikit-learn nltk --quiet

Importaciones

In [ ]:
import shap
import torch
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
# Crear el masker y explainer
masker = shap.maskers.Text(tokenizer=pipe.tokenizer)
explainer = shap.Explainer(pipe, masker)

# Aseguramos textos, convertimos a string y limitamos a 1000 caracteres (preparamos los cinco primeros registros)
sample_texts_shap = df['Text'].dropna().astype(str).apply(lambda x: x[:1000]).tolist()[:5]

# Obtener explicaciones
shap_values = explainer(sample_texts_shap)

In [ ]:
# Visualizar una explicación
shap.plots.text(shap_values[0], display=True)

Guardar la explicación SHAP como archivo HTML

In [ ]:
# Guardar como archivo HTML
html_explanation = shap.plots.text(shap_values[0], display=False)
with open("shap_explicacion_BETO.html", "w", encoding="utf-8") as f:f.write(html_explanation)

Gráfico de barras Tokens más influyentes (sin filtrar)

In [ ]:
# Obtener tokens y valores SHAP para la clase 1 ("fake")
tokens = shap_values[0].data
importancias = shap_values[0].values[:, 1]  # Tomamos la importancia para la clase 1

# Crear DataFrame
df_orden = pd.DataFrame({"Token": tokens, "SHAP": importancias})

# Seleccionar top 50 por magnitud del SHAP value
df_orden['SHAP_abs'] = df_orden['SHAP'].abs()
df_orden = df_orden.sort_values(by='SHAP_abs', ascending=False).head(50)

# Ordenar por magnitud absoluta (impacto)
df_orden["SHAP_abs"] = df_orden["SHAP"].abs()
df_ordenado = df_orden.sort_values(by="SHAP_abs", ascending=False)

# Seleccionar top 50 tokens más influyentes
top_n = 50
df_top = df_ordenado.head(top_n)

# Crear gráfico de barras
plt.figure(figsize=(16, 6))
bars = plt.bar(df_top["Token"], df_top["SHAP"], edgecolor='black')

# Colorear según signo
for bar, value in zip(bars, df_top["SHAP"]):
    bar.set_color('#1f77b4' if value > 0 else 'red')

plt.xticks(rotation=45, ha='right')
plt.title("Importancia atribuida por BETO (clase: fake) — filtrado")
plt.xlabel("Token")
plt.ylabel("Puntuación de importancia")
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

Gráfico de barras importancia FILTRADO

In [ ]:
import nltk
from nltk.corpus import stopwords

# Descargar stopwords
nltk.download('stopwords')
spanish_stopwords = set(stopwords.words('spanish'))

# Obtener tokens y valores SHAP para la clase 1 ("fake")
tokens = [t.strip() for t in shap_values[0].data]  # Limpieza: eliminar espacios extra
importancias = shap_values[0].values[:, 1]  # Importancia para clase 1

# Crear DataFrame
df = pd.DataFrame({
    "Token": tokens,
    "SHAP": importancias
})

# Filtrar:
df_filtrado = df[df["Token"].str.isalpha()]  # 1. Eliminar tokens que no son alfabéticos
df_filtrado = df_filtrado[~df_filtrado["Token"].str.lower().isin(spanish_stopwords)]  # 2. Eliminar stopwords
df_filtrado = df_filtrado[df_filtrado["Token"].str.len() >= 4]  # 3. Eliminar tokens con menos de 4 letras

# Ordenar por magnitud absoluta (impacto)
df_filtrado["SHAP_abs"] = df_filtrado["SHAP"].abs()
df_sorted = df_filtrado.sort_values(by="SHAP_abs", ascending=False)

# Seleccionar top 50 tokens más influyentes
top_n = 50
df_top = df_sorted.head(top_n)

# Crear gráfico de barras
plt.figure(figsize=(16, 6))
bars = plt.bar(df_top["Token"], df_top["SHAP"], edgecolor='black')

# Colorear según signo
for bar, value in zip(bars, df_top["SHAP"]):
    bar.set_color('#1f77b4' if value > 0 else 'red')

plt.xticks(rotation=45, ha='right')
plt.title("Importancia atribuida por BETO (clase: fake) — filtrado")
plt.xlabel("Token")
plt.ylabel("Puntuación de importancia")
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

Nube de palabras

In [ ]:
from wordcloud import WordCloud

# Crear diccionario de frecuencias
frequencies = dict(zip(df_top["Token"], df_top["SHAP_abs"]))

import random
random.seed(42)

# Función de color aleatorio
def random_color_func(word=None, font_size=None, position=None, orientation=None, font_path=None, random_state=None):
    return "hsl({}, 100%, 40%)".format(random.randint(0, 360))

# Crear y generar la nube
wc = WordCloud(width=800, height=400, background_color='white', color_func=random_color_func)
wc.generate_from_frequencies(frequencies)

# Mostrar nube
plt.figure(figsize=(12, 6))
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.title("Nube de palabras SHAP\n")
plt.tight_layout()
plt.show()

Tokens más influyentes FILTRADO (con valores exactos)

In [ ]:
def analizar_shap_explicacion2(shap_value, top_n=5, filtrar=True, class_index=None):

    # 1. Qué clase usar
    base_values = shap_value.base_values
    class_index = 1

    base_value = base_values[class_index] if isinstance(base_values, np.ndarray) else base_values

    # 2. Extraer valores SHAP de la clase activa
    shap_vals = shap_value.values
    if isinstance(shap_vals[0], np.ndarray):
        shap_vals = [v[class_index] for v in shap_vals]

    tokens = [t.strip() for t in shap_value.data]  # Eliminamos espacios extra
    total_contrib = np.sum(shap_vals)
    fx = base_value + total_contrib

    print("Análisis de predicción individual")
    print(f"Clase analizada: {class_index}")
    print(f"Valor base del modelo: {base_value:.3f}")
    print(f"Suma total de contribuciones (tokens): {total_contrib:.3f}")
    print(f"Predicción final (f(x)) ≈ {fx:.3f}")

    # 3. Crear DataFrame de tokens
    df = pd.DataFrame({
        "Token": tokens,
        "SHAP": shap_vals
    })

    # 4. Filtro
    if filtrar:
        df["Token"] = df["Token"].str.strip()  # limpieza adicional por seguridad
        df = df[df["Token"].str.isalpha()]  # Elimina puntuación y símbolos
        df = df[~df["Token"].str.lower().isin(spanish_stopwords)]  # Elimina stopwords
        df = df[df["Token"].str.len() >= 4]  # Elimina tokens muy cortos

    # 5. Ordenar por impacto absoluto
    df["SHAP_abs"] = df["SHAP"].abs()
    df_sorted = df.sort_values(by="SHAP_abs", ascending=False)

    # 6. Mostrar top_n tokens más influyentes
    top_tokens = df_sorted.head(top_n)

    print("\n Tokens más influyentes (positivo o negativo):")
    print(top_tokens[["Token", "SHAP"]].to_string(index=False))

    # 7. Conclusión
    if fx > 0.5:
        conclusion = "El modelo cree que esta noticia es FALSA (LABEL_1)."
    elif fx < 0.5:
        conclusion = "El modelo cree que esta noticia es VERDADERA (LABEL_0)."
    else:
        conclusion = "El modelo está indeciso entre FALSA y VERDADERA."

    print(f"\nConclusión del modelo: {conclusion}")

In [ ]:
analizar_shap_explicacion2(shap_values[0], top_n=10, filtrar=True, class_index=0)

Análisis de predicción individual (tokens que empujan hacia cada clase)

In [ ]:
def analizar_shap_explicacion(shap_value, top_n=5, filtrar=True):

    # 1. Qué clase usar
    base_values = shap_value.base_values
    class_index = 1

    base_value = base_values[class_index] if isinstance(base_values, np.ndarray) else base_values

    # 2. Extraer valores SHAP de la clase activa
    shap_vals = shap_value.values
    if isinstance(shap_vals[0], np.ndarray):
        shap_vals = [v[class_index] for v in shap_vals]

    tokens = [t.strip() for t in shap_value.data]  # Eliminamos espacios extra
    total_contrib = np.sum(shap_vals)
    fx = base_value + total_contrib

    print("Análisis de predicción individual")
    print(f"Clase analizada: {class_index}")
    print(f"Valor base del modelo: {base_value:.3f}")
    print(f"Suma total de contribuciones (tokens): {total_contrib:.3f}")
    print(f"Predicción final (f(x)) ≈ {fx:.3f}")

    # 3. Crear DataFrame de tokens
    df = pd.DataFrame({
        "Token": tokens,
        "SHAP": shap_vals
    })

    # 4. Filtro
    if filtrar:
        df["Token"] = df["Token"].str.strip()  # limpieza adicional por seguridad
        df = df[df["Token"].str.isalpha()]  # Elimina puntuación y símbolos
        df = df[~df["Token"].str.lower().isin(spanish_stopwords)]  # Elimina stopwords
        df = df[df["Token"].str.len() >= 4]  # Elimina tokens muy cortos


    # 5. Tokens más influyentes
    top_neg = df.sort_values(by="SHAP").head(top_n)
    top_pos = df.sort_values(by="SHAP", ascending=False).head(top_n)

    print("\n🔻 Tokens que empujan hacia la clase contraria:")
    print(top_neg.to_string(index=False))

    print("\n🔺 Tokens que empujan hacia esta clase:")
    print(top_pos.to_string(index=False))

    # 6. Resultado
    if fx > 0.5:
        conclusion = "El modelo cree que esta noticia es FALSA (LABEL_1)."
    elif fx < 0.5:
        conclusion = "El modelo cree que esta noticia es VERDADERA (LABEL_0)."
    else:
        conclusion = "El modelo está indeciso entre FALSA y VERDADERA."

    print(f"\nConclusión del modelo: {conclusion}")

In [ ]:
analizar_shap_explicacion(shap_values[0])



---


Tokens más influyentes: análisis global (sobre 100 registros aleatorios)

In [ ]:
# Usamos el dataset original con los textos
textos = df_texto['text'].dropna().astype(str).apply(lambda x: x[:1000]).tolist()

# Seleccionamos una muestra aleatoria
sample_texts = random.sample(textos, 100)

# Creamos el masker y el explainer si aún no están
masker = shap.maskers.Text(tokenizer=pipe.tokenizer)
explainer = shap.Explainer(pipe, masker)

# Aplicamos SHAP
shap_values = explainer(sample_texts)

Mostrar los tokens más influyentes en gráfico de barras

In [ ]:
import collections

# Acumulamos la contribución absoluta de cada token
token_contributions = collections.Counter()

for sv in shap_values:
    for token, value in zip(sv.data, sv.values):
        token_contributions[str(token)] += sum(abs(v) for v in value)

# Extraemos los 20 tokens más influyentes
top_tokens = token_contributions.most_common(20)

# Preparamos el gráfico
tokens, values = zip(*top_tokens)
plt.figure(figsize=(10, 6))
plt.barh(tokens[::-1], values[::-1])  # De menor a mayor
plt.xlabel("Importancia total (|SHAP|)")
plt.title("Tokens más influyentes según SHAP")
plt.tight_layout()
plt.show()

Eliminar stopwords en español

In [ ]:
import re

def limpiar_token(token):
    # Elimina todo lo que no sea letra (incluso espacios y signos) y pone en minúsculas
    return re.sub(r'[^a-zA-ZáéíóúñÁÉÍÓÚÑ]', '', token.lower().strip())


# Calcular contribuciones acumuladas
token_contributions = collections.Counter()
for sv in shap_values:
    for token, value in zip(sv.data, sv.values):
        token_contributions[str(token)] += sum(abs(v) for v in value)

# Filtrar: sin stopwords, sin símbolos, mínimo 4 letras
token_contributions_filtered = {
    token: score for token, score in token_contributions.items()
    if limpiar_token(token) not in spanish_stopwords and limpiar_token(token).isalpha() and len(limpiar_token(token)) >= 4
}

# Obtener los 20 tokens más influyentes
top_tokens = sorted(token_contributions_filtered.items(), key=lambda x: x[1], reverse=True)[:20]
tokens, scores = zip(*top_tokens)

# Mostrar gráfico
plt.figure(figsize=(10, 5))
plt.barh(tokens[::-1], scores[::-1])
plt.title("Tokens más influyentes sin stopwords (BETO - SHAP)")
plt.xlabel("Importancia acumulada (|SHAP|)")
plt.tight_layout()
plt.show()

Nube de palabras

In [ ]:
# Seleccionar top 20 tokens más influyentes
top_n = 20
top_tokens = sorted(token_contributions_filtered.items(), key=lambda x: x[1], reverse=True)[:top_n]

# Crear diccionario {token: valor absoluto de SHAP}
word_weights = {token: score for token, score in top_tokens}

# Función de color aleatorio en escala HSL
def random_color_func(word=None, font_size=None, position=None, orientation=None, font_path=None, random_state=None):
    return "hsl({}, 100%, 40%)".format(random.randint(0, 360))

# Crear y generar la nube de palabras
wc = WordCloud(
    width=1000,
    height=400,
    background_color='white',
    color_func=random_color_func,
    max_words=top_n
)

wc.generate_from_frequencies(word_weights)

# Mostrar la nube
plt.figure(figsize=(12, 6))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.title("Nube de palabras (importancia SHAP)\n", fontsize=14)
plt.tight_layout()
plt.show()

Gráficos por clase (tokens que empujan hacia Fake y tokens que empujan hacia True)

In [ ]:
# Contador para contribuciones hacia clase True (0) y clase False (1)
contrib_true = collections.Counter()
contrib_false = collections.Counter()

for sv in shap_values:
    for token, value in zip(sv.data, sv.values):
        contrib_true[str(token)] += value[0]   # Clase True
        contrib_false[str(token)] += value[1]  # Clase False

# Limpieza de tokens
contrib_true_cleaned = {
    token: score for token, score in contrib_true.items()
    if limpiar_token(token) not in spanish_stopwords and limpiar_token(token).isalpha() and len(limpiar_token(token)) >= 4
}

contrib_false_cleaned = {
    token: score for token, score in contrib_false.items()
    if limpiar_token(token) not in spanish_stopwords and limpiar_token(token).isalpha() and len(limpiar_token(token)) >= 4
}

# Mostrar tokens que más empujan hacia clase True
top_true = sorted(contrib_true_cleaned.items(), key=lambda x: x[1], reverse=True)[:20]
tokens_true, scores_true = zip(*top_true)

plt.figure(figsize=(10, 5))
plt.barh(tokens_true[::-1], scores_true[::-1])
plt.title("Tokens que más contribuyen a clase True")
plt.xlabel("SHAP (acumulado)")
plt.tight_layout()
plt.show()

# Mostrar tokens que más empujan hacia clase False
top_false = sorted(contrib_false_cleaned.items(), key=lambda x: x[1], reverse=True)[:20]
tokens_false, scores_false = zip(*top_false)

plt.figure(figsize=(10, 5))
plt.barh(tokens_false[::-1], scores_false[::-1])
plt.title("\nTokens que más contribuyen a clase False")
plt.xlabel("SHAP (acumulado)")
plt.tight_layout()
plt.show()

Nube de palabras bicolor (azul clase 0 - rojo clase 1)

In [ ]:
# Construir diccionario combinado para la nube (valores absolutos)
word_freqs = {}

for token, score in top_true:
    word_freqs[token] = abs(score)

for token, score in top_false:
    word_freqs[token] = abs(score)

# Crear diccionario de colores: azul para True, rojo para False
def color_func(word, *args, **kwargs):
    if word in dict(top_true):
        return "blue"
    elif word in dict(top_false):
        return "red"
    else:
        return "gray"  # fallback por si acaso

# Generar nube de palabras
wc = WordCloud(
    width=1000,
    height=400,
    background_color='white',
    color_func=color_func
)

wc.generate_from_frequencies(word_freqs)

# Mostrar nube
plt.figure(figsize=(16, 6))
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.title("Tokens más influyentes según dirección de la predicción (azul = True, rojo = False)\n", fontsize=14)
plt.tight_layout()
plt.show()



---



# Predicción manual con noticia nueva


In [ ]:
# === PREDICCIÓN MANUAL CON NOTICIA NUEVA ===


# Cargar modelo entrenado desde carpeta local
model_path = "./modelo_beto_fake_news"
tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
model = BertForSequenceClassification.from_pretrained(model_path)

# Pasar a modo evaluación
model.eval()

# Noticia de prueba
text = "Una nueva ley establece que todos los coches deberán ser eléctricos en 2035."

# Tokenizar
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

# Predicción
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class_id = torch.argmax(logits, dim=1).item()

# Interpretar resultado
etiquetas = ["Real", "Fake"]  # Adjusted labels to match expected output
print("Predicción:", etiquetas[predicted_class_id]) # Adjusted labels to match expected output

# También se puede hacer predicción manual iterando sobre los ejemplos uno a uno

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report
from tqdm import tqdm  # Para barra de progreso

# Nos aseguramos de que el modelo y tokenizer están cargados
model.eval()

# Listas para almacenar predicciones y etiquetas reales
y_true = []
y_pred = []

for text, label in tqdm(zip(test_df["text"], test_df["label"]), total=len(test_df)):
    # Tokenizar
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        pred_id = torch.argmax(logits, dim=1).item()

    y_true.append(label)
    y_pred.append(pred_id)

# === Resultados ===
print("Accuracy:", accuracy_score(y_true, y_pred))
print("F1-score:", f1_score(y_true, y_pred, average="weighted"))
print("\nReporte completo:\n", classification_report(y_true, y_pred, target_names=["Fake", "Real"]))



---



# Código para la interfaz interactiva

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr


# Asegúrate de que el modelo y tokenizer están cargados
model.eval()
etiquetas = ["Real", "Fake"]

def clasificar_noticia(texto):
    inputs = tokenizer(texto, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        pred_id = torch.argmax(logits, dim=1).item()
    return f"Predicción: {etiquetas[pred_id]}"

# Interfaz con Gradio
demo = gr.Interface(
    fn=clasificar_noticia,
    inputs=gr.Textbox(lines=6, placeholder="Escribe o pega aquí una noticia..."),
    outputs="text",
    title="Detector de Noticias Fake con BETO",
    description="Introduce una noticia y el modelo clasificará si es FAKE o REAL."
)

# Lanza la app
demo.launch()